# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
#setup imports
# imports
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama
import gradio as gr

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")
if api_key and api_key.startswith(('sk-proj-', 'sk-or-v1-')) and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")


API key looks good so far


In [66]:
# craft system prompt for AI technical asistance that explain technical questions concisely
system_prompt = """
 You are a customer technical support assistant at fintech compnay called XYZ. 
 Your primary objective is to provide precise, and easy to understand explanation to a given technical questions about their tranasactions.

Behavioural constraints
- Prefer correctness over verbosity.
- Ask clarification questions when requirements are underspecified.
- Do not invent missing facts.
- Explicitly state uncertainty.

Reasoning Approach:
- Analyze requirements before answering.
- Break complex problems into steps.
- State assumptions when needed.

Output Requirements:
- Use structured formatting.
- Provide concrete recommendations.
- Avoid speculation.
- Be concise but complete.
- Be Polite

Interaction Guidelines:
- Ask clarification questions when ambiguity exists.
- Do not fabricate information.
- Maintain consistency across conversation turns.
"""

In [5]:
# lets setup tool functions and mock data

transaction_db = {
    "TXN-USD-0001": {"amount": 7000, "currency": "USD", "status": "processing"},
    "TXN-USD-0002": {"amount": 1250, "currency": "USD", "status": "successful"},
    "TXN-USD-0003": {"amount": 980, "currency": "USD", "status": "failed"},
    "TXN-USD-0004": {"amount": 4300, "currency": "USD", "status": "pending"},
    "TXN-USD-0005": {"amount": 250, "currency": "USD", "status": "successful"},
    "TXN-USD-0006": {"amount": 9999, "currency": "USD", "status": "processing"},
    "TXN-USD-0007": {"amount": 3100, "currency": "USD", "status": "failed"},
    "TXN-USD-0008": {"amount": 15000, "currency": "USD", "status": "pending"},
    "TXN-USD-0009": {"amount": 875, "currency": "USD", "status": "successful"},
    "TXN-USD-0010": {"amount": 6400, "currency": "USD", "status": "processing"},

    "TXN-NGN-0001": {"amount": 500000, "currency": "NGN", "status": "successful"},
    "TXN-NGN-0002": {"amount": 120000, "currency": "NGN", "status": "processing"},
    "TXN-NGN-0003": {"amount": 75000, "currency": "NGN", "status": "failed"},
    "TXN-NGN-0004": {"amount": 900000, "currency": "NGN", "status": "pending"},
    "TXN-NGN-0005": {"amount": 30000, "currency": "NGN", "status": "successful"},
    "TXN-NGN-0006": {"amount": 450000, "currency": "NGN", "status": "processing"},
    "TXN-NGN-0007": {"amount": 210000, "currency": "NGN", "status": "failed"},
    "TXN-NGN-0008": {"amount": 66000, "currency": "NGN", "status": "pending"},
    "TXN-NGN-0009": {"amount": 880000, "currency": "NGN", "status": "successful"},
    "TXN-NGN-0010": {"amount": 150000, "currency": "NGN", "status": "processing"},

    "TXN-GBP-0001": {"amount": 450, "currency": "GBP", "status": "successful"},
    "TXN-GBP-0002": {"amount": 1200, "currency": "GBP", "status": "processing"},
    "TXN-GBP-0003": {"amount": 800, "currency": "GBP", "status": "failed"},
    "TXN-GBP-0004": {"amount": 2500, "currency": "GBP", "status": "pending"},
    "TXN-GBP-0005": {"amount": 150, "currency": "GBP", "status": "successful"},
    "TXN-GBP-0006": {"amount": 990, "currency": "GBP", "status": "processing"},
    "TXN-GBP-0007": {"amount": 1750, "currency": "GBP", "status": "failed"},
    "TXN-GBP-0008": {"amount": 300, "currency": "GBP", "status": "pending"},
    "TXN-GBP-0009": {"amount": 640, "currency": "GBP", "status": "successful"},
    "TXN-GBP-0010": {"amount": 1100, "currency": "GBP", "status": "processing"}
}

def get_transaction(reference):
    print(f"Getting transaction for {reference}")
    transaction = transaction_db.get(reference.upper(), "Transaction Not found")
    return str(transaction)




In [6]:
# setup AI tools using the functions

get_transaction_function = {
    "name": "get_transaction",
'description': "Get a single transaction details",
"parameters": {
    "type": "object",
    "properties": {
        "reference": {
            "type": 'string',
            "description": "transaction unique reference id"
        },
    },
    "required": ["reference"],
    "additionalProperties": False
}
}

fintech_tools = [{"type": "function","function": get_transaction_function}]

fintech_tools

[{'type': 'function',
  'function': {'name': 'get_transaction',
   'description': 'Get a single transaction details',
   'parameters': {'type': 'object',
    'properties': {'reference': {'type': 'string',
      'description': 'transaction unique reference id'}},
    'required': ['reference'],
    'additionalProperties': False}}}]

In [36]:
#tool call handler

#handle multiple tool calls

def handle_fintech_tool_calls(message):
    responses = []

    for tool_call in message['tool_calls']:
        tool_function = globals().get(tool_call.function.name, lambda: None)
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('reference')
        trx = tool_function(city)
        responses.append({
                "role": "tool",
                "content": trx,
                "tool_call_id": tool_call.id
            })
    return responses

In [55]:
# Customer technical suppor chat function for gradio
def customer_support_chat(message, history, model):

    if not message:
        return history

    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    provider = None

    if(model == MODEL_GPT):
        openrouter_url = "https://openrouter.ai/api/v1"
        provider = OpenAI(base_url=openrouter_url)
    else:
        ollama_url="http://localhost:11434/v1"
        provider = OpenAI(base_url=ollama_url, api_key="ollama")


    response = provider.chat.completions.create(model= model, messages=messages, tools=fintech_tools)

    responseChoice = response.choices
    print(responseChoice)
    while response.choices[0].finish_reason == 'tool_calls':
        message = response.choices[0].message
        toolsResponses = handle_fintech_tool_calls(message)
        messages.append(message)
        messages.extend(toolsResponses)
        response = provider.chat.completions.create(model= model, messages=messages, tools=fintech_tools)

        print(response, "response now in a tool call")

    return response.choices[0].message.content



In [ ]:
#enable streaming
# Customer technical suppor chat function for gradio
def stream_customer_support_chat(message, history, model):

    if not message:
        yield ""
        return ""

    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    provider = None

    if(model == MODEL_GPT):
        openrouter_url = "https://openrouter.ai/api/v1"
        provider = OpenAI(base_url=openrouter_url)
    else:
        ollama_url="http://localhost:11434/v1"
        provider = OpenAI(base_url=ollama_url, api_key="ollama")

    stream = provider.chat.completions.create(model= model, messages=messages, tools=fintech_tools, stream=True)

    response = ""
    tool_call_data = []
    isToolCall = False
    
    for chunk in stream:
        chunkChoice = chunk.choices[0]
        chunkChoiceDelta = chunkChoice.delta

        for tool_call in chunkChoiceDelta.tool_calls or []:
            if tool_call:
                idx = tool_call.index
                if  not tool_call_data[idx: idx+1]:
                    tool_call_data.append(tool_call)
                
                if  tool_call.id:
                    tool_call_data[tool_call.index].id = tool_call.id or ''

                if  tool_call.function.name:
                    tool_call_data[tool_call.index].function.name = tool_call.function.name
                
                if  tool_call.function.arguments:
                    tool_call_data[tool_call.index].function.arguments += tool_call.function.arguments

        response += chunkChoiceDelta.content or ''
        yield response

        if chunkChoice.finish_reason == 'tool_calls':
            isToolCall = True
    
    if  isToolCall:
        message = {'role': 'assistant', 'content': response, 'tool_calls': tool_call_data }
        toolsResponses = handle_fintech_tool_calls(message)
        messages.append(message)
        messages.extend(toolsResponses)
        tool_call_stream = provider.chat.completions.create(model= model, messages=messages, tools=fintech_tools, stream=True)
        for tool_chunk in tool_call_stream:
            response += tool_chunk.choices[0].delta.content or ''
            yield response

    return response



In [61]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [65]:
# additonal input to chatInterface to select model
# 2. Setup the Additional Inputs
model_selector = gr.Dropdown(
    choices=[MODEL_GPT, MODEL_LLAMA], 
    value= MODEL_GPT,
    label="Select Model",
    info="Pick from the available models"
)

def validateInput(input):
    return [
        gr.validate(not input, "Please provide input that I can help you with"),
    ]

gr.ChatInterface(fn=stream_customer_support_chat, type="messages", additional_inputs=[model_selector], validator=validateInput).launch()

* Running on local URL:  http://127.0.0.1:7886
* To create a public link, set `share=True` in `launch()`.


True message
